# 단어의 의미 파악하기

딥러닝이 사용되기 이전에 어떤 방식으로 단어의 의미를 파악했는지 알아보자.

## 시소러스(Thesaurus)

> 정의: 단어를 의미에 따라 분류·배열한 일종의 유의어 사전

시소러스는 단어 사이의 상하 관계나 포함 관계를 고려하여 정의할 수도 있다. (이 경우엔 그래프로 표현)

단점: 시대 변화에 대응이 어렵다. 많은 인력이 필요하다. 단어의 미묘한 차이를 표현할 수 없다.

WordNet: 1985년부터 프린스턴 대학교에서 구축하기 시작한 가장 유명한 시소러스

## 통계 기반 기법

- **말뭉치(corpus)**: 수집된 텍스트 데이터. 추가 정보가 포함되는 경우가 있다(품사, 긍정 부정 여부 등).

말뭉치를 전처리하여 쉽게 다룰 수 있도록 코드를 작성해보겠다.

In [1]:
text = 'you say goodbye and I say hello.'
text = text.lower()
text = text.replace('.', ' .')
text

'you say goodbye and i say hello .'

In [2]:
words = text.split()
words

['you', 'say', 'goodbye', 'and', 'i', 'say', 'hello', '.']

단어를 효율적으로 조작하기 위해 id를 부여한다.

In [3]:
word_to_id = {}
id_to_word = {}

for word in words:
    new_id = len(word_to_id)
    word_to_id[word] = new_id
    id_to_word[new_id] = word

In [4]:
id_to_word

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}

In [5]:
word_to_id

{'you': 0, 'say': 5, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}

말뭉치 전처리 함수를 정의한다.

In [25]:
import numpy as np

def preprocess(t):
    text = t.lower()
    text = text.replace('.', ' .')
    words = text.split()
    word_to_id = {}
    id_to_word = {}

    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word

    corpus = np.array([word_to_id[w] for w in words])
    
    return  corpus, word_to_id, id_to_word

### 단어의 분산 표현

우리는 색을 3차원 벡터(RGB)로 표현한다. 데이터로 표현하기도 쉽고 색 끼리의 관련성을 판단하기도 용이하다. 

이처럼 단어도 벡터 공간 상에 표현할 수 있다. 이를 단어의 **분산표현(distributional representation)** 이라고 한다.

### 분포 가설

> 단어의 의미는 주변 단어에 의해 형성된다. 단어 그 자체에는 의미가 없다. 단어가 사용된 맥락(context)이 의미를 형성한다.

이는 분포 가설(distributional hypothesis)의 핵심이다. 

- 맥락(context)

    특정 단어를 중심에 둔 그 주변 단어를 의미한다.
    
    - 맥락의 크기 = window size
    
        ex) window size가 2이면 좌측 두 단어, 우측 두 단어가 맥락에 포함된다.
        
### 동시발생 행렬

> 어떤 단어의 맥락이 되는 다른 단어의 출현 빈도를 행렬로 표현한 것

In [27]:
corpus, word_to_id, id_to_word = preprocess(text)
corpus, word_to_id, id_to_word

(array([0, 1, 2, 3, 4, 1, 5, 6]),
 {'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6},
 {0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'})

In [28]:
def create_co_matrix(corpus, vocab_size, window_size=1):
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)
    
    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i
            
            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1
                
            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1
    
    return co_matrix

In [32]:
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

In [60]:
import pandas as pd

def draw_color_at_nan(x, color):
    if x == 1:
        color = f'background-color:{color}'
        return color
    else:
        return ''

index = [id_to_word[i] for i in range(vocab_size)]

df = pd.DataFrame(C, index=index, columns=index)
df.style.applymap(draw_color_at_nan, color='#a0c0ff')

,you,say,goodbye,and,i,hello,.
you,0,1,0,0,0,0,0
say,1,0,1,0,1,1,0
goodbye,0,1,0,1,0,0,0
and,0,0,1,0,1,0,0
i,0,1,0,1,0,0,0
hello,0,1,0,0,0,0,1
.,0,0,0,0,0,1,0


### 벡터 간 유사도

단어 벡터 사이의 유사도를 나타낼 때 **코사인 유사도(cosine similarity)** 를 자주 이용한다.

$$\text{similarity}(\textbf{x}, \textbf{y}) = cos(\theta) = \frac{\textbf{x} \cdot \textbf{y}}{\lVert \textbf{x} \rVert \lVert \textbf{y} \rVert} = \frac{x_1y_1+ \cdots +x_ny_n}{\sqrt{x_1^2+ \cdots + x_n^2}\sqrt{y_1^2+ \cdots + y_n^2}}$$

$\theta$는 두 벡터 $\textbf{x}, \textbf{y}$ 가 이루는 각도이다.

In [68]:
def cos_similarity(x, y, eps=1e-8):
    nx = x / (np.sqrt(np.sum(x**2)) + eps)
    ny = x / (np.sqrt(np.sum(y**2)) + eps)
    return np.dot(nx, ny)

"you"와 "i"의 유사도를 측정한다.

In [69]:
c0 = C[word_to_id['you']]
c1 = C[word_to_id['i']]
print(cos_similarity(c0, c1))                

0.7071067691154799


In [ ]:
co